# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 3. Обработка текста и облака слов

### Часть 1: подготовка к работе

В этом практикуме мы будем работать с текстами новостей науки, которые мы ранее научились выгружать со страницы сайта [nplus1.ru](https://nplus1.ru/). В частности, построим на основе текстов облака слов (*word clouds*), которые показывают, какие слова встречаются в тексте чаще, а какие – реже.

Установим необходимые библиотеки:

* библиотека `wordcloud` для построения облака слов ([тьюториал](https://www.datacamp.com/tutorial/wordcloud-python) по работе с библиотекой);
* библиотека `pymorphy2` для морфологического анализа текстов, понадобится для приведения слов к начальной форме ([документация](https://pymorphy2.readthedocs.io/en/stable/)). 

In [ ]:
!pip install --upgrade pip
!pip install --upgrade wordcloud 
!pip install --upgrade pymorphy2 

Команда `pip install ...`  используется для установки библиотек, к ней можно добавить опцию `--upgrade` на случай, если библиотека на компьютере уже установлена в более старой версии, а мы хотим её обновить. Символ `!` в начале строки сообщает Jupyter, что это особая операция, как будто бы запускаемая с командной строки или из терминала, а не код Python с соответствующим синтаксисом. Строка с установкой `pip` в самом начале нужна для обновления самого установщика `pip` (если версия установщика старая, логично, что он не все новые версии библиотек сможет корректно поставить).

Импортируем библиотеки, чтобы убедиться, что всё установилось (для надёжности можно перезапустить ядро через *Kernel - Restart* и после импортировать):

In [ ]:
import wordcloud
import pymorphy2

Проверили, теперь давайте импортируем из библиотек отдельные функции (глобальный импорт выше можно в дальнейшем пропускать и импортировать только отдельные функции):

* функция (класс, кто знаком с ООП в Python) `WordCloud` для создания объекта типа *облако слов*;
* функция (тоже класс) `MorphAnalyzer` для создания объекта типа *морфологический анализатор*;
* модуль `pyplot` из библиотеки `matplotlib` для построения графиков.

In [ ]:
from wordcloud import WordCloud
from pymorphy2 import MorphAnalyzer
from matplotlib import pyplot as plt

Проверим, что всё работает, на маленьком тексте:

In [ ]:
# фрагмент из «Макбета» У.Шекспира
# всё в нижнем регистре без знаков препинания

test = """
кто вы ответьте если речь дана вам
хвала тебе макбет гламисский тан
хвала тебе макбет кавдорский тан
хвала макбету королю в грядущем
"""

Давайте разобьём текст на отдельные слова и приведём их к единой начальной форме! Так, все существительные будут стоять в единственном числе в именительном падеже, все прилагательные – тоже в единственном числе в именительном падеже, но ещё и в мужском роде, глаголы – в неопределённой форме. Эта процедура называется **лемматизацией** (*лемма* – словарная форма слова). Иногда для унификации слов используют другую процедуру – **стемминг** (от английского *stem* – основа), которая предполагает отсечение всех формообразующих морфем вроде окончаний и суффиксов глаголов, но для нашей цели она не подходит – слова в облаке слов будут некрасиво обрублены.

Разбиваем текст на слова:

In [ ]:
words = test.split()
print(words)

Создаем объект типа «морфологический анализатор», он поможет автоматически сделать морфологический разбор слова и забрать его начальную форму:

In [ ]:
morph = MorphAnalyzer()

Посмотрим на работу анализатора на примере одного слова (разбор слова или предложения в обработке текста и лингвистике тоже называется парсинг, так как парсинг вообще – это автоматический разбор какой-то структуры):

In [ ]:
morph.parse("стекла")

Метод `.parse()` вернул список всех возможных вариантов разбора слова, упорядоченный от наиболее вероятного до наименее вероятного. Так, с самой большой вероятностью слово «стекла» – это родительный падеж слова «стекло» (`NOUN` – существительное, `inan` – неодушевленное, `sing` – единственное число, `gent` – родительный падеж, генитив). А с самой маленькой вероятностью это слово является формой глагола «стечь» (`VERB` – глагол, `perf` – прошедшее время, `intr` – непереходный глагол, `indc` – изъявительное наклонение, индикатив).

Из списка можно извлечь самый вероятный первый разбор и забрать его начальную форму:

In [ ]:
morph.parse("стекла")[0].normal_form

Задействуем списковое включение и получим для каждого слова в `test` начальную форму:

In [ ]:
words_norm = [morph.parse(w)[0].normal_form for w in words]
print(words_norm)

Теперь снова склеим слова в единый текст – для облака слов список не подойдёт:

In [ ]:
test_norm = " ".join(words_norm)
print(test_norm)

Построим облако слов с помощью `WordCloud()`:

In [ ]:
# wcloud – объект, в котором хранится информация для построения облака
# временно соханен в какой-то ячейке памяти

wcloud = WordCloud().generate(test_norm)
wcloud

Если выводятся странные ошибки, попробуйте установить более старую версию библиотеки `Pillow`, от неё зависят некоторые процедуры в `wordcloud`: `!pip install Pillow==9.5.0`.

Теперь отрисуем полученное облако через функцию `.imshow()` и «выключим» оси, так как метки с числами (как на обычных графиках) нам не нужны:

In [ ]:
plt.imshow(wcloud)
plt.axis("off")
plt.show()

Проблема: облако слов получилось неинформативным! В нём, действительно, собраны самые частые слова вроде союзов, а мы явно хотели не этого. На самом деле, наша предварительная обработка текста была выполнена не до конца – мы забыли убрать так называемые **стоп-слова**, самые частые слова в языке, которые обычно исключаются перед анализом текстов. Вариантов добыть стоп-слова для русского языка много: можно подключить специальные бибиотеки вроде `nltk`, можно загрузить список из репозитория какого-нибудь проекта на Github или из готового файла. 

Для этого маленького текста мы просто сформируем список стоп-слов вручную:

In [ ]:
stop = ["кто", "ты", "вы", "если"]

# аргумент stopwords внутри WordCloud()

wcloud = WordCloud(stopwords = stop).generate(test_norm)
plt.imshow(wcloud)
plt.axis("off")
plt.show()

Итак, простые облака слов мы строить научились, перейдём к более интересным текстам новостей.

### Часть 2: облака слов для новостей

Импортируем библиотеку `pandas` и считаем данные из csv-файла с выгруженными ранее новостями науки:

In [ ]:
import pandas as pd

In [ ]:
news = pd.read_csv("nplus1_upd.csv")
news.head()

Выберем столбец `text` с текстами новостей и склеим все новости в единый текст:

In [ ]:
full = "".join(news["text"])

Приведём всё к нижнему регистру (маленьким буквам):

In [ ]:
full = full.lower()

Уберём знаки препинания – воспользуемся готовым перечнем из модуля `string`:

In [ ]:
from string import punctuation

In [ ]:
print(punctuation)

Объект `punctuation` – самая обычная строка со знаками препинания. Доклеим к ней недостающие символы – длинное тире и русские кавычки «ёлочки» (да, это вполне официальное название, английские кавычки называются «лапками»):

In [ ]:
punctuation = punctuation + "—" + "«»" 

Теперь осуществим замену – заменим все знаки препинания в тексте (из `punctuation`) на пробелы:

In [ ]:
for p in punctuation:
    full = full.replace(p, " ")
    
# фрагмент текста
print(full[0:98])

Дальнейшую обработку текста проведём по аналогии с предыдущим примером.

### Задача 1

Разбейте текст `full` на список слов и приведите все слова к начальной форме с помощью `pymorphy2`. Сохраните текст, состоящий из слов в начальной форме, в переменную `full_norm`.

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Загрузите из файла `stop_words_russian.txt` стоп-слова и сохраните их в список `stop_ru`. 

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Постройте облако слов для текста `full_norm` так, чтобы стоп-слова из `stop_ru` в облако не были внесены. 

In [ ]:
### YOUR CODE HERE ###

### Задача 4

Напишите функцию `prepare_for_cloud()`, которая принимает на вход строку с текстом для визуализации, а возвращает строку с обработанным текстом (текст в нижнем регистре, без знаков препинания, слова в начальной форме).

In [ ]:
### YOUR CODE HERE ###

### Часть 3: немного интерактива и настройка дизайна облака слов

Давайте добавим фильтр на рубрику новостей – запросим значение с клавиатуры у пользователя:

In [ ]:
rub = input()

Отфильтруем соответствующие строки из таблицы:

In [ ]:
chosen = news[news["rubrics"].str.contains(rub)]

### Задача 5

Постройте облако слов для новостей выбранной рубрики, используя функцию `prepare_for_cloud()` для предварительной обработки текста. Сделайте фон белым, а в качестве цветовой палитры используйте встроенную палитру `magma`.

In [ ]:
### YOUR CODE HERE ###

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9), dpi = 300) 

### YOUR CODE HERE ###

В качестве основы для облака слов можно взять изображение, это прекрасно описано в тьюториале [здесь](https://www.datacamp.com/tutorial/wordcloud-python). Подобрать какое-то узнаваемое и простое по форме тематическое изображение в данном случае сложновато, поэтому давайте просто возьмём векторное изображение с [Freepik](https://ru.freepik.com/).

Импортируем функцию для обработки изображения и библиотеку `numpy`, чтобы потом преобразовать изображение в числовой массив:

In [ ]:
from PIL import Image
import numpy as np

Загружаем изображение и создаём массив:

In [ ]:
my_mask = np.array(Image.open("42517.jpg"))

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9), dpi = 300) 

### YOUR CODE HERE ###

# mask = my_mask